In [ ]:
!pip install streamlit
!pip install -qU python-dotenv
!pip install -qU langchain_core
!pip install -qU langchain_mistralai

In [ ]:
import os
os.environ["MISTRAL_API_KEY"]="YOUR API KEY HERE"

In [13]:
%%writefile interview_assistant.py
import os
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
load_dotenv()
class InterviewAssistant:
  def __init__(self):
    self.llm = ChatMistralAI(
        model="mistral-large-latest",
        api_key=os.getenv("MISTRAL_API_KEY"),
        temperature=0.1,
        max_retries=2,
    )


  def interview_process_by_assitant(self,question,history):

        prompt = ChatPromptTemplate.from_messages([
            (
                "system",
                """Your are the hr assitant for talentScount you will be assit for inizial screening.Your task is gather information from candidate one by one and one of the information[First show the details that could be ask then ask one by one] is tech stack ask 3 or 4 technical questions to canditate based on tech stack and ask technical questions one by one analyze the converstation histroy for maintain correct flow woth candidate.Once the inizial screening process completely done ending greeting with we cantact throw your email for the canditate.After the process candidate give any ending grettings like Thankyou response to this

                Information you need to gather:
                  Full Name
                  Email Address
                  Phone Number
                  Years of Experience
                  Desired Position(s)
                  Current Location
                  Tech Stack


                Return Format Should be like this:
                      {{
                        "full_name": "string",
                        "email": "string",
                        "phone_number": "string",
                        "years_of_experience": "string",
                        "desired_position": "string",
                        "current_location": "string",
                        "tech_stack": "string"
                        "technical_questions_and_answers": [{{"question1":"answer1","question2":"answer2","question3":"answer3"}}]
                      }}

                Example for technical question based on tech stack:
                  example tech stack: programming languages, frameworks, databases, and tools they are proficient in.
                  you need ask technical questions from this skills one by one

                Thinks to be reminder:
                    analyse history for maintain correct flow with candidate
                    first[staring] show what are the information you need ask then ask one by one information
                    provide meaningful responses do not have string like this "[Provide your answer or name...]
                    once all the process completed give end greetings.
                    We follow all GDPR guidelines to protect your privacy so candidate not intersted to share this information give end greetings.
                """,
            ),
            ("human", "{candidate_input}  {history}",)
        ])
        chain = prompt | self.llm
        llm_response=chain.invoke(
                {
                    "candidate_input": question,
                    "history": history,
                }
            )

        result=llm_response.content
        return result

Overwriting interview_assistant.py


In [12]:
%%writefile app.py
import re
import streamlit as st
from interview_assistant import InterviewAssistant

interview_assistant=InterviewAssistant()


st.header("TalentScout Interview Assistant")
if "messages" not in st.session_state:
    st.session_state.messages = []

if "inisial_message" not in st.session_state:
    st.session_state.inisial_message = []

with st.chat_message("assistant"):
    msg_to_candidate="To assist you better, we kindly request your name, email, and phone number. Your information will be handled securely and will not be shared with anyone. We follow all GDPR guidelines to protect your privacy. Do you agree to share this information?"
    st.markdown(msg_to_candidate)
    st.session_state.inisial_message.append({"role": "assistant", "content": msg_to_candidate})

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])


if prompt := st.chat_input("What is up?"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    response=interview_assistant.interview_process_by_assitant(prompt,[st.session_state.inisial_message,st.session_state.messages])

    pattern = r'(\{\s*"full_name":.*\})'

    match = re.search(pattern, response, re.DOTALL)

    if match:
        extracted_json = match.group(1)
        print(extracted_json)

    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 --subdomain interview-assistant